In [24]:
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import dill
from typing import List
import scipy.stats as sts
import pickle
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint

import util
from pathlib import Path
from sk.replace_column_dataframe import ReplaceColumnDataFrame
from sk.norm_standard_scaler import NormStandardScaler
import constants.columns_dataframe as const
from class_manipulates_path import ManipulatePath
from class_preprocessing_refactor import Preprocessing
from class_format_data import FormatData
util.init()

current_path = sys.path[0]

In [25]:
import tensorflow as tf

# Verifique se a GPU está disponível
print("GPU disponível:", tf.config.list_physical_devices('GPU'))

# Verifique se o TensorFlow está usando a GPU
print("TensorFlow está usando a GPU:", tf.test.is_built_with_cuda())

GPU disponível: []
TensorFlow está usando a GPU: False


In [26]:
print("Configurações do TensorFlow relacionadas à GPU:")
print(tf.config.list_physical_devices('GPU'))
print("TensorFlow está usando a GPU:", tf.test.is_built_with_cuda())
print("Número de GPUs disponíveis:", len(tf.config.experimental.list_physical_devices('GPU')))

Configurações do TensorFlow relacionadas à GPU:
[]
TensorFlow está usando a GPU: False
Número de GPUs disponíveis: 0


In [27]:
def plot_variable(df, column_name, intervalos=None):
    """
    Plota um gráfico de dispersão para uma variável de um DataFrame.

    Parâmetros:
        - df: DataFrame pandas contendo os dados.
        - column_name: Nome da coluna que você deseja plotar.
        - intervalos: Lista de intervalos para plotar linhas verticais.
    """
    plt.figure(figsize=(20, 10))
    plt.scatter(df.index.values, df[column_name].values, color='darkcyan', alpha=0.5)
    plt.xlabel('Índice')
    plt.ylabel(column_name)
    plt.title(f'Gráfico de dispersão para a variável "{column_name}"')


    # Plotar linhas verticais para cada intervalo
    if intervalos:
        for intervalo in intervalos:
            plt.axvline(x=intervalo, linestyle='--', color='red')

    plt.show()

In [28]:
def mapping_labels(df: pd.DataFrame):
    df_data = df.copy()
    unique_labels = df_data['class'].unique()
    unique_labels.sort()

    # Mapear os rótulos para inteiros em ordem crescente
    label_mapping = {label: i for i, label in enumerate(unique_labels)}

    # Aplicar o mapeamento aos rótulos verdadeiros
    labels_int = df_data['class'].map(label_mapping)

    # Criar DataFrame com rótulos inteiros
    df_mapped = pd.DataFrame({'class': df_data['class'], 'mapped_class': labels_int})
    df_data['class'] = df_mapped["mapped_class"]
    return df_data, label_mapping

def inverse_mapping_labels(df: pd.DataFrame, label_mapping: dict):
    df_data = df.copy()
    # Inverter o mapeamento original
    inverse_label_mapping = {v: k for k, v in label_mapping.items()}

    # Aplicar o mapeamento inverso aos rótulos
    labels_original = df_data['class'].map(inverse_label_mapping)

    # Atualizar o DataFrame com os rótulos originais
    df_data['class'] = labels_original
    return df_data

In [29]:
manipulate_path = ManipulatePath()

preprocessing = Preprocessing()

format_data = FormatData()

path_raw_data = manipulate_path.get_path_raw_data()

In [30]:
df_preprocessing = pd.read_parquet(manipulate_path.get_path_preprocessing_real_data_all_classes())

In [31]:
contagem_classes = df_preprocessing['class'].value_counts()

# Calcular a porcentagem de cada classe
porcentagem_classes_real = contagem_classes / len(df_preprocessing) * 100
porcentagem_classes_real

class
0.0    32.679781
5.0    30.983203
7.0    29.009351
1.0     4.797527
6.0     1.810310
2.0     0.719828
Name: count, dtype: float64

Aqui é possível ver que tem poquíssimas classes normais!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# Rede 1

In [32]:
df_preprocessing, label_mapping = mapping_labels(df_preprocessing)

In [33]:
label_mapping

{0.0: 0, 1.0: 1, 2.0: 2, 5.0: 3, 6.0: 4, 7.0: 5}

In [34]:
contagem_classes = df_preprocessing['class'].value_counts()

# Calcular a porcentagem de cada classe
porcentagem_classes_real = contagem_classes / len(df_preprocessing) * 100
porcentagem_classes_real

class
0    32.679781
3    30.983203
5    29.009351
1     4.797527
4     1.810310
2     0.719828
Name: count, dtype: float64

In [35]:
# Separar as features (X) e os rótulos (y)
X = df_preprocessing.drop('class', axis=1)
y = df_preprocessing['class']

In [36]:
X

,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP
timestamp,,,,
2014-01-24 09:33:03,18433410.0,116.8718,9397031.0,74.80031
2014-01-24 09:33:04,18433410.0,116.8718,9397031.0,74.80031
2014-01-24 09:33:05,18433410.0,116.8718,9397032.0,74.80031
2014-01-24 09:33:06,18433410.0,116.8718,9397033.0,74.80031
2014-01-24 09:33:07,18433410.0,116.8718,9397033.0,74.80031
...,...,...,...,...
2019-04-03 14:59:56,8489354.0,109.7213,1496222.0,73.38219
2019-04-03 14:59:57,8489349.0,109.7212,1495828.0,73.38310
2019-04-03 14:59:58,8489344.0,109.7210,1495433.0,73.38401


In [37]:
X_train_k, X_test, y_train_k, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [38]:
X_train, X_val, y_train, y_val = train_test_split(X_train_k, y_train_k, test_size=0.2, stratify=y_train_k, random_state=42)

In [39]:
scaler = NormStandardScaler(X_train.columns)
X_train_norm = scaler.fit_transform(X_train)

X_train_k_norm = scaler.transform(X_train_k)

X_val_norm = scaler.transform(X_val)

X_test_norm = scaler.transform(X_test)

In [40]:
# with open('scaler.pkl', 'wb') as file:
#     pickle.dump(scaler, file)

In [41]:
from sklearn.model_selection import KFold
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

In [43]:
accuracies = []
histories = []

for train_index, val_index in kf.split(X_train_k_norm):
    # Divida os dados em treino e validação
    X_train_fold, X_val_fold = X_train_k_norm.iloc[train_index], X_train_k_norm.iloc[val_index]
    y_train_fold, y_val_fold = y_train_k.iloc[train_index], y_train_k.iloc[val_index]

    model_2 = Sequential()

    # Adicione as camadas ocultas
    model_2.add(Dense(32, input_dim=4, activation='relu'))  # Camada de entrada com 8 neurônios e ativação ReLU
    model_2.add(Dense(16, activation='relu'))  # Segunda camada oculta com 3 neurônios e ativação ReLU (opcional)
    model_2.add(Dense(8, activation='relu'))

    # Adicione a camada de saída
    model_2.add(Dense(6, activation='softmax'))  # Camada de saída com 5 neurônios (um para cada classe) e ativação Softmax para classificação multiclasse

    # Compile o model_2o
    model_2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Defina o checkpoint para salvar os pesos
    checkpoint = ModelCheckpoint("weights_only.h5", monitor='val_loss', save_best_only=True)

    # Treine o model_2o com conjunto de validação e o checkpoint
    history = model_2.fit(X_train_fold, y_train_fold, epochs=100, batch_size=64, validation_data=(X_val_fold, y_val_fold), callbacks=[checkpoint])
    histories.append(history.history)

    # Avalie o modelo
    _, accuracy = model_2.evaluate(X_val_fold, y_val_fold, verbose=0)
    accuracies.append(accuracy)
    print(f'Acurácia do fold: {accuracy*100:.2f}%')

Epoch 1/100
8556/8556 [==============================] - 12s 1ms/step - loss: 0.3243 - accuracy: 0.8746 - val_loss: 0.1911 - val_accuracy: 0.9256
Epoch 2/100
 135/8556 [..............................] - ETA: 9s - loss: 0.1964 - accuracy: 0.9243

c:\Users\marce\Documents\0_python_env\venv3WPetrobras\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8556/8556 [==============================] - 12s 1ms/step - loss: 0.1718 - accuracy: 0.9344 - val_loss: 0.1528 - val_accuracy: 0.9431
Epoch 3/100
8556/8556 [==============================] - 12s 1ms/step - loss: 0.1371 - accuracy: 0.9444 - val_loss: 0.1285 - val_accuracy: 0.9502
Epoch 4/100
8556/8556 [==============================] - 12s 1ms/step - loss: 0.1190 - accuracy: 0.9513 - val_loss: 0.1085 - val_accuracy: 0.9523
Epoch 5/100
8556/8556 [==============================] - 13s 1ms/step - loss: 0.1096 - accuracy: 0.9561 - val_loss: 0.1051 - val_accuracy: 0.9555
Epoch 6/100
8556/8556 [==============================] - 13s 2ms/step - loss: 0.1025 - accuracy: 0.9589 - val_loss: 0.1010 - val_accuracy: 0.9606
Epoch 7/100
8556/8556 [==============================] - 12s 1ms/step - loss: 0.0976 - accuracy: 0.9607 - val_loss: 0.1043 - val_accuracy: 0.9588
Epoch 8/100
8556/8556 [==============================] - 12s 1ms/step - loss: 0.0943 - accuracy: 0.9619 - val_loss: 0.0934 - val_accurac

c:\Users\marce\Documents\0_python_env\venv3WPetrobras\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8556/8556 [==============================] - 12s 1ms/step - loss: 0.1639 - accuracy: 0.9373 - val_loss: 0.1446 - val_accuracy: 0.9526
Epoch 3/100
8556/8556 [==============================] - 12s 1ms/step - loss: 0.1360 - accuracy: 0.9476 - val_loss: 0.1269 - val_accuracy: 0.9490
Epoch 4/100
8556/8556 [==============================] - 11s 1ms/step - loss: 0.1241 - accuracy: 0.9511 - val_loss: 0.1290 - val_accuracy: 0.9455
Epoch 5/100
8556/8556 [==============================] - 12s 1ms/step - loss: 0.1175 - accuracy: 0.9531 - val_loss: 0.1188 - val_accuracy: 0.9520
Epoch 6/100
8556/8556 [==============================] - 11s 1ms/step - loss: 0.1131 - accuracy: 0.9539 - val_loss: 0.1041 - val_accuracy: 0.9558
Epoch 7/100
8556/8556 [==============================] - 12s 1ms/step - loss: 0.1107 - accuracy: 0.9542 - val_loss: 0.1188 - val_accuracy: 0.9475
Epoch 8/100
8556/8556 [==============================] - 11s 1ms/step - loss: 0.1071 - accuracy: 0.9556 - val_loss: 0.1116 - val_accurac

...

8556/8556 [==============================] - 12s 1ms/step - loss: 0.1548 - accuracy: 0.9376 - val_loss: 0.1368 - val_accuracy: 0.9555
Epoch 3/100
8556/8556 [==============================] - 12s 1ms/step - loss: 0.1302 - accuracy: 0.9466 - val_loss: 0.1173 - val_accuracy: 0.9497
Epoch 4/100
8556/8556 [==============================] - 12s 1ms/step - loss: 0.1192 - accuracy: 0.9503 - val_loss: 0.1179 - val_accuracy: 0.9455
Epoch 5/100
8556/8556 [==============================] - 13s 1ms/step - loss: 0.1127 - accuracy: 0.9530 - val_loss: 0.1274 - val_accuracy: 0.9523
Epoch 6/100
8556/8556 [==============================] - 12s 1ms/step - loss: 0.1083 - accuracy: 0.9550 - val_loss: 0.1014 - val_accuracy: 0.9584
Epoch 7/100
8556/8556 [==============================] - 13s 1ms/step - loss: 0.1047 - accuracy: 0.9565 - val_loss: 0.1084 - val_accuracy: 0.9525
Epoch 8/100
8556/8556 [==============================] - 12s 1ms/step - loss: 0.1002 - accuracy: 0.9580 - val_loss: 0.0976 - val_accurac

c:\Users\marce\Documents\0_python_env\venv3WPetrobras\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8556/8556 [==============================] - 15s 2ms/step - loss: 0.1669 - accuracy: 0.9370 - val_loss: 0.1639 - val_accuracy: 0.9353
Epoch 3/100
8556/8556 [==============================] - 10s 1ms/step - loss: 0.1377 - accuracy: 0.9461 - val_loss: 0.1260 - val_accuracy: 0.9440
Epoch 4/100
8556/8556 [==============================] - 11s 1ms/step - loss: 0.1191 - accuracy: 0.9497 - val_loss: 0.1071 - val_accuracy: 0.9543
Epoch 5/100
8556/8556 [==============================] - 10s 1ms/step - loss: 0.1097 - accuracy: 0.9531 - val_loss: 0.1059 - val_accuracy: 0.9591
Epoch 6/100
8556/8556 [==============================] - 10s 1ms/step - loss: 0.1055 - accuracy: 0.9548 - val_loss: 0.1136 - val_accuracy: 0.9486
Epoch 7/100
8556/8556 [==============================] - 10s 1ms/step - loss: 0.1014 - accuracy: 0.9570 - val_loss: 0.0980 - val_accuracy: 0.9609
Epoch 8/100
8556/8556 [==============================] - 11s 1ms/step - loss: 0.0983 - accuracy: 0.9585 - val_loss: 0.0982 - val_accurac

c:\Users\marce\Documents\0_python_env\venv3WPetrobras\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8556/8556 [==============================] - 12s 1ms/step - loss: 0.2106 - accuracy: 0.9022 - val_loss: 0.1894 - val_accuracy: 0.9055
Epoch 3/100
8556/8556 [==============================] - 12s 1ms/step - loss: 0.1793 - accuracy: 0.9214 - val_loss: 0.1590 - val_accuracy: 0.9459
Epoch 4/100
8556/8556 [==============================] - 12s 1ms/step - loss: 0.1503 - accuracy: 0.9401 - val_loss: 0.1288 - val_accuracy: 0.9479
Epoch 5/100
8556/8556 [==============================] - 13s 2ms/step - loss: 0.1310 - accuracy: 0.9468 - val_loss: 0.1267 - val_accuracy: 0.9521
Epoch 6/100
8556/8556 [==============================] - 13s 1ms/step - loss: 0.1199 - accuracy: 0.9510 - val_loss: 0.1255 - val_accuracy: 0.9431
Epoch 7/100
8556/8556 [==============================] - 13s 1ms/step - loss: 0.1122 - accuracy: 0.9537 - val_loss: 0.1095 - val_accuracy: 0.9569
Epoch 8/100
8556/8556 [==============================] - 12s 1ms/step - loss: 0.1072 - accuracy: 0.9557 - val_loss: 0.0997 - val_accurac

In [44]:
mean_accuracy = np.mean(accuracies)
std_accuracy = np.std(accuracies)
print(f'\nAcurácia média: {mean_accuracy*100:.2f}%')
print(f'Desvio padrão da acurácia: {std_accuracy*100:.2f}%')


Acurácia média: 98.04%
Desvio padrão da acurácia: 0.28%


In [45]:
import json

# Salvar accuracies em um arquivo JSON
with open('accuracies_rede_unica.json', 'w') as f:
    json.dump(accuracies, f)

# Salvar histories em um arquivo JSON
with open('histories_rede_unica.json', 'w') as f:
    json.dump(histories, f)

In [21]:
#model_2 = Sequential()
#
## Adicione as camadas ocultas
#model_2.add(Dense(12, input_dim=4, activation='relu'))  # Camada de entrada com 8 neurônios e ativação ReLU
#model_2.add(Dense(12, activation='relu'))  # Segunda camada oculta com 3 neurônios e ativação ReLU (opcional)
#model_2.add(Dense(8, activation='relu'))
#model_2.add(Dense(8, activation='relu'))
#
## Adicione a camada de saída
#model_2.add(Dense(6, activation='softmax'))  # Camada de saída com 5 neurônios (um para cada classe) e ativação Softmax para classificação multiclasse
#
## Compile o model_2o
#model_2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#
## Defina o checkpoint para salvar os pesos
#checkpoint = ModelCheckpoint("weights_only.h5", monitor='val_loss', save_best_only=True)
#
## Treine o model_2o com conjunto de validação e o checkpoint
#model_2.fit(X_train_norm, y_train, epochs=100, batch_size=10, validation_data=(X_val_norm, y_val), callbacks=[checkpoint])

Epoch 1/100
47909/47909 [==============================] - 91s 2ms/step - loss: 0.2546 - accuracy: 0.8913 - val_loss: 0.1717 - val_accuracy: 0.9265
Epoch 2/100
   99/47909 [..............................] - ETA: 1:14 - loss: 0.1617 - accuracy: 0.9273

c:\Users\marce\Documents\0_python_env\venv3WPetrobras\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


47909/47909 [==============================] - 89s 2ms/step - loss: 0.1313 - accuracy: 0.9457 - val_loss: 0.1137 - val_accuracy: 0.9572
Epoch 3/100
47909/47909 [==============================] - 88s 2ms/step - loss: 0.1192 - accuracy: 0.9501 - val_loss: 0.1028 - val_accuracy: 0.9584
Epoch 4/100
47909/47909 [==============================] - 85s 2ms/step - loss: 0.1158 - accuracy: 0.9517 - val_loss: 0.1248 - val_accuracy: 0.9490
Epoch 5/100
47909/47909 [==============================] - 78s 2ms/step - loss: 0.1116 - accuracy: 0.9532 - val_loss: 0.1281 - val_accuracy: 0.9459
Epoch 6/100
47909/47909 [==============================] - 78s 2ms/step - loss: 0.1083 - accuracy: 0.9545 - val_loss: 0.1037 - val_accuracy: 0.9573
Epoch 7/100
47909/47909 [==============================] - 79s 2ms/step - loss: 0.1052 - accuracy: 0.9559 - val_loss: 0.0959 - val_accuracy: 0.9583
Epoch 8/100
47909/47909 [==============================] - 78s 2ms/step - loss: 0.1026 - accuracy: 0.9567 - val_loss: 0.0872

In [46]:
y_pred = model_2.predict(X_test_norm)
y_pred = np.argmax(y_pred, axis=1)

accuracy_score(y_test, y_pred)

9167/9167 [==============================] - 7s 719us/step


0.9807957179871812

In [47]:
precision = precision_score(y_test, y_pred, average=None)
recall = recall_score(y_test, y_pred, average=None)
f1 = f1_score(y_test, y_pred, average=None)
for i in range(len(precision)):
    print(f'Classe {np.unique(y_test)[i]}:')
    print(f'Precision: {precision[i]}')
    print(f'Recall: {recall[i]}')
    print(f'F1-score: {f1[i]}\n')

Classe 0:
Precision: 0.9643059315143219
Recall: 0.9774140377232515
F1-score: 0.970815739709349

Classe 1:
Precision: 1.0
Recall: 1.0
F1-score: 1.0

Classe 2:
Precision: 1.0
Recall: 0.9919507575757576
F1-score: 0.9959591157594485

Classe 3:
Precision: 0.9891425694459646
Recall: 0.9944322183098592
F1-score: 0.9917803408579613

Classe 4:
Precision: 0.9925182122465053
Recall: 0.9493408662900188
F1-score: 0.9704495139089422

Classe 5:
Precision: 0.9864151575722031
Recall: 0.968550946057116
F1-score: 0.9774014314601012

